In [43]:
import pandas as pd
from calitp_data_analysis.sql import query_sql
from shared_utils import gtfs_utils_v2

# Explore Thruway Bus (and state-suppoted rail) data availability

In [35]:
analysis_date = '2025-08-18'

## New SJJPA feed

In [2]:
query = '''
SELECT * from cal-itp-data-infra.mart_transit_database.dim_gtfs_service_data
WHERE _is_current
LIMIT 1000
'''

In [3]:
df = query_sql(query)

In [4]:
sanj = df.query('name.str.contains("Amtrak San J")')

In [5]:
sanj

,key,name,source_record_id,service_key,gtfs_dataset_key,customer_facing,category,fares_v2_status,manual_check__fixed_route_completeness,manual_check__demand_response_completeness,_is_current,_valid_from,_valid_to
716,c9364f7e1d226a2a1b57de51ec4375ad,Amtrak San Joaquins – Amtrak San Joaquins Sche...,recBvVy7cJzjXBjeD,b3a26733da02debeb0e23e30c8d8e928,524ea6209600e9a2de34a02cf9068729,True,primary,[Unknown],Unknown,Unknown,True,2025-08-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00


In [44]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [47]:
sanj = feeds.query('name.str.contains("Amtrak San Joaquins")')
sanj

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,regional_feed_type,type
81,a04bf510313ab925aa37eea637a18bec,2025-08-18,d9e1e77d0754b712fc608741ae3836f5,America/Los_Angeles,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,524ea6209600e9a2de34a02cf9068729,Amtrak San Joaquins Schedule,None,schedule


In [48]:
gtfs_utils_v2.get_shapes?

Signature:
gtfs_utils_v2.get_shapes(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    shape_cols: list[str] = [],
    get_df: bool = True,
    crs: str = 'EPSG:4326',
    custom_filtering: dict = None,
) -> geopandas.geodataframe.GeoDataFrame
Docstring:
Query fct_daily_scheduled_shapes.

Must supply a list of feed_keys returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [56]:
shape_cols = ['feed_key', 'shape_array_key']

In [57]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [58]:
sanj_shapes

,feed_key,shape_array_key,geometry
0,d9e1e77d0754b712fc608741ae3836f5,da0c6c7161bfec6a9e1f593818fa716f,"LINESTRING (-121.73805 38.54396, -121.73816 38..."
1,d9e1e77d0754b712fc608741ae3836f5,e23ce2c052c42d45cfb9cc5dda4581d3,"LINESTRING (-119.00893 35.37317, -119.00894 35..."
2,d9e1e77d0754b712fc608741ae3836f5,20629525c84b0d3ac8da45c65c35b845,"LINESTRING (-119.78328 36.73841, -119.78321 36..."
3,d9e1e77d0754b712fc608741ae3836f5,07088e4adf69e4f231ea4ba11914b887,"LINESTRING (-121.50064 38.58416, -121.49956 38..."
4,d9e1e77d0754b712fc608741ae3836f5,15e2f4643843159420d0e1b1563d7046,"LINESTRING (-119.84824 34.41522, -119.84819 34..."
...,...,...,...
63,d9e1e77d0754b712fc608741ae3836f5,ef9cbdb19e2f885fb77ba9da12ed3d3e,"LINESTRING (-122.29181 37.84045, -122.29161 37..."
64,d9e1e77d0754b712fc608741ae3836f5,82db3c1be8f398c6f49d9db438eaa443,"LINESTRING (-122.39850 37.78850, -122.39832 37..."
65,d9e1e77d0754b712fc608741ae3836f5,e2c04f958d1a6a3ab444b5590924ad27,"LINESTRING (-119.00770 35.37218, -119.00762 35..."
66,d9e1e77d0754b712fc608741ae3836f5,4d1d764f99b6f3abaecd0ae238776088,"LINESTRING (-122.27147 37.79379, -122.27169 37..."


In [55]:
gtfs_utils_v2.get_trips?

Signature:
gtfs_utils_v2.get_trips(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    trip_cols: list[str] = [],
    get_df: bool = True,
    custom_filtering: dict = None,
) -> Union[pandas.core.frame.DataFrame, siuba.sql.backend.LazyTbl]
Docstring:
Query fct_scheduled_trips

Must supply a list of feed_keys returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [68]:
trip_cols = ['trip_instance_key', 'trip_id', 'trip_short_name',
'base64_url', 'feed_key', 'name',
'regional_feed_type', 'gtfs_dataset_key', 'service_date',
'direction_id', 'block_id', 'route_key',
'route_id', 'route_type', 'route_short_name',
'route_long_name', 'route_desc', 'agency_id',
'network_id', 'shape_array_key', 'shape_id'
]

In [69]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=trip_cols)

In [85]:
# sanj_trips.query('route_type == "2"')

In [63]:
sanj_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

,,trip_instance_key
route_short_name,route_long_name,
ACETrain,ACE Train,8
AMSJ2,Amtrak San Joaquins BFD OKJ,12
Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - Oceanside- San Diego,20
Route 10,Santa Barbara - Las Vegas,2
Route 15,Mammoth Lakes- Yosemite National Park,4
Route 15a,Merced - Yosemite National Park,12
Route 15b,Fresno - Yosemite National Park,6
Route 18,Santa Maria - San Luis Obispo - Handford,2
Route 19,Berkersfield - Inland Empire,4


In [64]:
sanj_shapes_trip_info = sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [89]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [73]:
sanj_trips.query('route_short_name == "Route 99"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
57,bfa4ea4f854d9bac2537be504ee0e9e4,6615,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
58,97c347c91d8733658a03d0ba680cb470,6617,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
59,f34ce074ba43c8d8326cbbf80ef9f57d,6619,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
60,9d9dae8395b78521dc2067ffc4fee6cd,6611,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
61,eeed34ed58303fa1c5d4f1065b9a571e,6613,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
83,75bb0a711792d09aac78643b7dd46cd6,6614,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
84,978012028f8316de61c45e5ec928f705,6668_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
85,b2a88bb09ebd891178e8d29e4fe97a31,6610,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
86,963b3d03c6e386332149c6e8afe740b2,6612_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
87,c0b3951c7daedca3c122bb2423d5a41b,6616,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu


## Capitol Corridor

In [78]:
cc_feed = feeds.query('name.str.contains("Capitol")')

In [79]:
cc_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=cc_feed.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [80]:
cc_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=cc_feed.feed_key, trip_cols=trip_cols)

In [81]:
cc_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

,,trip_instance_key
route_short_name,route_long_name,
CC,Capitol Corridor,28
SF,Shuttle_SF,26
Shuttle,Shuttle_Auburn,10


In [82]:
cc_shapes_trip_info = cc_shapes.merge(cc_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [88]:
# cc_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [86]:
sanj_trips.query('route_short_name == "Route 99"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
57,bfa4ea4f854d9bac2537be504ee0e9e4,6615,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
58,97c347c91d8733658a03d0ba680cb470,6617,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
59,f34ce074ba43c8d8326cbbf80ef9f57d,6619,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
60,9d9dae8395b78521dc2067ffc4fee6cd,6611,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
61,eeed34ed58303fa1c5d4f1065b9a571e,6613,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
83,75bb0a711792d09aac78643b7dd46cd6,6614,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
84,978012028f8316de61c45e5ec928f705,6668_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
85,b2a88bb09ebd891178e8d29e4fe97a31,6610,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
86,963b3d03c6e386332149c6e8afe740b2,6612_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
87,c0b3951c7daedca3c122bb2423d5a41b,6616,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu


In [87]:
cc_trips.query('route_short_name == "SF"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
18,f0f222080b1336524d379fb2874244f5,546E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
19,f6eedf567dba77205bd5ead0d9716743,542E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
20,ab06192193a09902515cf96951962568,532E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
21,d534a5ce51c715361dd84aaf57c85f01,540E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
22,5bd052d717ed43fa5b0bdf5dfeec3bb2,534E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
23,64c31d10efcedd508506596436b74184,544E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
24,fe9f54f0a3842f6d960c2dd37f5e51f1,536E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
25,039c9338d38c428e2fd747688b2942eb,538E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
26,a6acc60c7c41d4dc3c2dd778ffaa243b,528E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902cb81cb24,cpz2
27,a24b008f6f61f2f4a6a1475af0be141c,524E,None,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,d8ed633646e8cca3020de0933411eda4,Bay Area 511 Capitol Corridor Schedule,Regional Subfeed,f5a749dd65924e025b1293c58f95f8d6,2025-08-18,0,...,dc98d265d521a9eda6808bcdc520ddb1,SF,3,SF,Shuttle_SF,Shuttle to San Francisco Transbay Terminal,AM,None,e4888477d2d7cd0c6b951902